# Product Search Retrieval 
## Methods Implemented
1. **TF-IDF (Baseline)** - traditional term frequency approach
2. **BM25** - probabilistic ranking function that improves on TF-IDF
3. **Semantic Embeddings** - uses sentence transformers to capture semantic meaning
4. **Hybrid (RRF)** - combines BM25 and semantic methods using Reciprocal Rank Fusion

## Evaluation Metrics
- **MAP@10** - Mean Average Precision (primary metric)
- **SoftMAP** - gives partial credit for near-misses
- **nDCG@10** - Normalized Discounted Cumulative Gain

---

In [31]:
# ===================================================================
# Install required packages
# ===================================================================
!pip install -q rank-bm25 sentence-transformers

# Note: pandas, scikit-learn, and numpy should already be installed
# If needed, install with: pip install pandas scikit-learn numpy


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer

In [3]:
#clone the git repo that contains the data and additional information about the dataset
!git clone https://github.com/wayfair/WANDS.git

fatal: destination path 'WANDS' already exists and is not an empty directory.


In [4]:
#define functions for product search using Tf-IDF
def calculate_tfidf(dataframe):
    """
    Calculate the TF-IDF for combined product name and description.

    Parameters:
    dataframe (pd.DataFrame): DataFrame with product_id, and other product information.

    Returns:
    TfidfVectorizer, csr_matrix: TF-IDF vectorizer and TF-IDF matrix.
    """
    # Combine product name and description to vectorize
    # NOTE: Please feel free to use any combination of columns available, some columns may contain NULL values
    combined_text = dataframe['product_name'] + ' ' + dataframe['product_description']
    vectorizer = TfidfVectorizer()
    # convert combined_text to list of unicode strings
    tfidf_matrix = vectorizer.fit_transform(combined_text.values.astype('U'))
    return vectorizer, tfidf_matrix

def get_top_products(vectorizer, tfidf_matrix, query, top_n=10):
    """
    Get top N products for a given query based on TF-IDF similarity.

    Parameters:
    vectorizer (TfidfVectorizer): Trained TF-IDF vectorizer.
    tfidf_matrix (csr_matrix): TF-IDF matrix for the products.
    query (str): Search query.
    top_n (int): Number of top products to return.

    Returns:
    list: List of top N product IDs.
    """
    query_vector = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_product_indices = cosine_similarities.argsort()[-top_n:][::-1]
    return top_product_indices

In [5]:
# ===================================================================
# BM25 retrieval functions
# ===================================================================

def tokenize_text(text):
    """simple tokenization by splitting on whitespace and lowercasing"""
    return str(text).lower().split()

def get_top_products_bm25(bm25, query, top_n=10):
    """
    retrieve top products using BM25 scoring
    
    bm25: BM25Okapi instance trained on product corpus
    query: search query string  
    top_n: number of results to return
    """
    tokenized_query = tokenize_text(query)
    scores = bm25.get_scores(tokenized_query)
    top_indices = np.argsort(scores)[-top_n:][::-1]
    return top_indices

In [6]:
# ===================================================================
# Semantic embedding retrieval functions
# ===================================================================

def get_top_products_semantic(model, product_embeddings, query, top_n=10):
    """
    retrieve top products using semantic similarity
    
    model: SentenceTransformer model
    product_embeddings: precomputed embeddings for all products
    query: search query string
    top_n: number of results to return
    """
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, product_embeddings).flatten()
    top_indices = np.argsort(similarities)[-top_n:][::-1]
    return top_indices

In [7]:
# ===================================================================
# Hybrid retrieval using Reciprocal Rank Fusion (RRF)
# ===================================================================

def reciprocal_rank_fusion(rankings_list, k=60, top_n=10):
    """
    combine multiple ranking lists using RRF
    
    rankings_list: list of ranking arrays (each contains product indices)
    k: constant for RRF formula (typically 60)
    top_n: number of final results to return
    
    RRF formula: score(d) = sum(1 / (k + rank(d))) across all rankings
    """
    rrf_scores = {}
    
    for rankings in rankings_list:
        for rank, product_idx in enumerate(rankings):
            if product_idx not in rrf_scores:
                rrf_scores[product_idx] = 0
            rrf_scores[product_idx] += 1.0 / (k + rank + 1)
    
    sorted_products = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    top_indices = [idx for idx, score in sorted_products[:top_n]]
    return top_indices

In [8]:
#define functions for evaluating retrieval performance
def map_at_k(true_ids, predicted_ids, k=10):
    """
    Calculate the Mean Average Precision at K (MAP@K).

    Parameters:
    true_ids (list): List of relevant product IDs.
    predicted_ids (list): List of predicted product IDs.
    k (int): Number of top elements to consider.
             NOTE: IF you wish to change top k, please provide a justification for choosing the new value

    Returns:
    float: MAP@K score.
    """
    #if either list is empty, return 0
    if not len(true_ids) or not len(predicted_ids):
        return 0.0

    score = 0.0
    num_hits = 0.0

    for i, p_id in enumerate(predicted_ids[:k]):
        if p_id in true_ids and p_id not in predicted_ids[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(true_ids), k)

In [9]:
# ===================================================================
# Additional evaluation metrics
# ===================================================================

def soft_map_at_k(true_ids, predicted_ids, k=10, tolerance=3):
    """
    Soft MAP gives partial credit for near-misses
    
    tolerance: how many positions away from exact match still counts
    """
    if not len(true_ids) or not len(predicted_ids):
        return 0.0
    
    score = 0.0
    num_hits = 0.0
    
    for i, p_id in enumerate(predicted_ids[:k]):
        if p_id in true_ids:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
        else:
            # check if any true_id appears within tolerance window
            found_nearby = False
            for j in range(max(0, i - tolerance), min(len(predicted_ids), i + tolerance + 1)):
                if j != i and predicted_ids[j] in true_ids:
                    found_nearby = True
                    break
            if found_nearby:
                num_hits += 0.5  # partial credit
                score += num_hits / (i + 1.0)
    
    return score / min(len(true_ids), k)


def ndcg_at_k(true_ids, predicted_ids, k=10):
    """
    Normalized Discounted Cumulative Gain at K
    
    measures ranking quality with position-based discounting
    """
    if not len(true_ids) or not len(predicted_ids):
        return 0.0
    
    # calculate DCG (actual ranking)
    dcg = 0.0
    for i, p_id in enumerate(predicted_ids[:k]):
        if p_id in true_ids:
            # relevance = 1 for exact match, 0 otherwise
            dcg += 1.0 / np.log2(i + 2)  # i+2 because positions start at 1, and log2(1)=0
    
    # calculate IDCG (ideal ranking - all relevant docs at top)
    idcg = 0.0
    for i in range(min(len(true_ids), k)):
        idcg += 1.0 / np.log2(i + 2)
    
    if idcg == 0:
        return 0.0
    
    return dcg / idcg

In [10]:
# get search queries
query_df = pd.read_csv("WANDS/dataset/query.csv", sep='\t')

In [11]:
query_df.head()

,query_id,query,query_class
0,0,salon chair,Massage Chairs
1,1,smart coffee table,Coffee & Cocktail Tables
2,2,dinosaur,Kids Wall Décor
3,3,turquoise pillows,Accent Pillows
4,4,chair and a half recliner,Recliners


In [12]:
# get products
product_df = pd.read_csv("WANDS/dataset/product.csv", sep='\t')

In [13]:
product_df.head()

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0


In [14]:
# get manually labeled groundtruth lables
label_df = pd.read_csv("WANDS/dataset/label.csv", sep='\t')

In [15]:
label_df.head()

,id,query_id,product_id,label
0,0,0,25434,Exact
1,1,0,12088,Irrelevant
2,2,0,42931,Exact
3,3,0,2636,Exact
4,4,0,42923,Exact


In [16]:
#group the labels for each query to use when identifying exact matches
grouped_label_df = label_df.groupby('query_id')

In [17]:
# Calculate TF-IDF
vectorizer, tfidf_matrix = calculate_tfidf(product_df)

In [18]:
# ===================================================================
# Build BM25 index
# ===================================================================

# tokenize product corpus for BM25
combined_text = product_df['product_name'] + ' ' + product_df['product_description']
combined_text = combined_text.fillna('')  # handle any null values
tokenized_corpus = [tokenize_text(doc) for doc in combined_text]

# initialize BM25
bm25 = BM25Okapi(tokenized_corpus)
print(f"BM25 index built with {len(tokenized_corpus)} products")

BM25 index built with 42994 products


In [19]:
# ===================================================================
# Load semantic model and compute embeddings
# ===================================================================

# using a lightweight but effective model
model = SentenceTransformer('all-MiniLM-L6-v2')

# generate embeddings for all products
product_texts = combined_text.tolist()
print("Computing embeddings for all products (this may take a minute)...")
product_embeddings = model.encode(product_texts, show_progress_bar=True)
print(f"Generated embeddings with shape: {product_embeddings.shape}")

Computing embeddings for all products (this may take a minute)...


Batches:   0%|          | 0/1344 [00:00<?, ?it/s]

Generated embeddings with shape: (42994, 384)


In [20]:
#Sanity check code block to see if the search results are relevant
#implementing a function to retrieve top K product IDs for a query
def get_top_product_ids_for_query(query):
    top_product_indices = get_top_products(vectorizer, tfidf_matrix, query, top_n=10)
    top_product_ids = product_df.iloc[top_product_indices]['product_id'].tolist()
    return top_product_ids

#define the test query
query = "armchair"

#obtain top product IDs
top_product_ids = get_top_product_ids_for_query(query)

print(f"Top products for '{query}':")
for product_id in top_product_ids:
    product = product_df.loc[product_df['product_id'] == product_id]
    print(product_id, product['product_name'].values[0])

Top products for 'armchair':
12756 24.41 '' wide tufted polyester armchair
42698 donham armchair
42697 donham 25 '' wide armchair
41270 almaraz 33.7 '' wide leather match armchair
23907 faizah 27.6 '' wide tufted polyester armchair
31564 biloxi 34.75 '' wide armchair
41306 hartsell 33 '' wide armchair
1527 howington 39 '' wide tufted linen armchair
42802 donham polyester lounge chair
6532 ogan 29 '' wide polyester armchair


In [21]:
#implementing a function to retrieve exact match product IDs for a query_id
def get_exact_matches_for_query(query_id):
    query_group = grouped_label_df.get_group(query_id)
    exact_matches = query_group.loc[query_group['label'] == 'Exact']['product_id'].values
    return exact_matches

#applying the function to obtain top product IDs and adding top K product IDs to the dataframe 
query_df['top_product_ids'] = query_df['query'].apply(get_top_product_ids_for_query)

#adding the list of exact match product_IDs from labels_df
query_df['relevant_ids'] = query_df['query_id'].apply(get_exact_matches_for_query)

#now assign the map@k score
query_df['map@k'] = query_df.apply(lambda x: map_at_k(x['relevant_ids'], x['top_product_ids'], k=10), axis=1)


In [22]:
# calculate the MAP across the entire query set
query_df.loc[:, 'map@k'].mean()

np.float64(0.293186576829806)

In [23]:
# ===================================================================
# BM25 rankings
# ===================================================================

def get_top_product_ids_bm25(query):
    top_indices = get_top_products_bm25(bm25, query, top_n=10)
    return product_df.iloc[top_indices]['product_id'].tolist()

query_df['bm25_top_product_ids'] = query_df['query'].apply(get_top_product_ids_bm25)
query_df['bm25_map@10'] = query_df.apply(
    lambda x: map_at_k(x['relevant_ids'], x['bm25_top_product_ids'], k=10), axis=1
)

print(f"BM25 MAP@10: {query_df['bm25_map@10'].mean():.4f}")

BM25 MAP@10: 0.3439


In [24]:
# ===================================================================
# Semantic rankings
# ===================================================================

def get_top_product_ids_semantic(query):
    top_indices = get_top_products_semantic(model, product_embeddings, query, top_n=10)
    return product_df.iloc[top_indices]['product_id'].tolist()

query_df['semantic_top_product_ids'] = query_df['query'].apply(get_top_product_ids_semantic)
query_df['semantic_map@10'] = query_df.apply(
    lambda x: map_at_k(x['relevant_ids'], x['semantic_top_product_ids'], k=10), axis=1
)

print(f"Semantic MAP@10: {query_df['semantic_map@10'].mean():.4f}")

Semantic MAP@10: 0.3569


In [25]:
# ===================================================================
# Hybrid rankings using RRF (combines BM25 + Semantic)
# ===================================================================

def get_top_product_ids_hybrid(query):
    bm25_indices = get_top_products_bm25(bm25, query, top_n=10)
    semantic_indices = get_top_products_semantic(model, product_embeddings, query, top_n=10)
    
    # combine using RRF
    hybrid_indices = reciprocal_rank_fusion([bm25_indices, semantic_indices], k=60, top_n=10)
    return product_df.iloc[hybrid_indices]['product_id'].tolist()

query_df['hybrid_top_product_ids'] = query_df['query'].apply(get_top_product_ids_hybrid)
query_df['hybrid_map@10'] = query_df.apply(
    lambda x: map_at_k(x['relevant_ids'], x['hybrid_top_product_ids'], k=10), axis=1
)

print(f"Hybrid (RRF) MAP@10: {query_df['hybrid_map@10'].mean():.4f}")

Hybrid (RRF) MAP@10: 0.3649


In [26]:
# ===================================================================
# Calculate additional metrics for all methods
# ===================================================================

# SoftMAP for all methods
query_df['tfidf_softmap'] = query_df.apply(
    lambda x: soft_map_at_k(x['relevant_ids'], x['top_product_ids'], k=10), axis=1
)
query_df['bm25_softmap'] = query_df.apply(
    lambda x: soft_map_at_k(x['relevant_ids'], x['bm25_top_product_ids'], k=10), axis=1
)
query_df['semantic_softmap'] = query_df.apply(
    lambda x: soft_map_at_k(x['relevant_ids'], x['semantic_top_product_ids'], k=10), axis=1
)
query_df['hybrid_softmap'] = query_df.apply(
    lambda x: soft_map_at_k(x['relevant_ids'], x['hybrid_top_product_ids'], k=10), axis=1
)

# nDCG@10 for all methods
query_df['tfidf_ndcg@10'] = query_df.apply(
    lambda x: ndcg_at_k(x['relevant_ids'], x['top_product_ids'], k=10), axis=1
)
query_df['bm25_ndcg@10'] = query_df.apply(
    lambda x: ndcg_at_k(x['relevant_ids'], x['bm25_top_product_ids'], k=10), axis=1
)
query_df['semantic_ndcg@10'] = query_df.apply(
    lambda x: ndcg_at_k(x['relevant_ids'], x['semantic_top_product_ids'], k=10), axis=1
)
query_df['hybrid_ndcg@10'] = query_df.apply(
    lambda x: ndcg_at_k(x['relevant_ids'], x['hybrid_top_product_ids'], k=10), axis=1
)

print("Additional metrics calculated for all methods")

Additional metrics calculated for all methods


In [27]:
# ===================================================================
# Results comparison across all methods
# ===================================================================

results = pd.DataFrame({
    'Method': ['TF-IDF (Baseline)', 'BM25', 'Semantic Embeddings', 'Hybrid (RRF)'],
    'MAP@10': [
        query_df['map@k'].mean(),
        query_df['bm25_map@10'].mean(),
        query_df['semantic_map@10'].mean(),
        query_df['hybrid_map@10'].mean()
    ],
    'SoftMAP': [
        query_df['tfidf_softmap'].mean(),
        query_df['bm25_softmap'].mean(),
        query_df['semantic_softmap'].mean(),
        query_df['hybrid_softmap'].mean()
    ],
    'nDCG@10': [
        query_df['tfidf_ndcg@10'].mean(),
        query_df['bm25_ndcg@10'].mean(),
        query_df['semantic_ndcg@10'].mean(),
        query_df['hybrid_ndcg@10'].mean()
    ]
})

print("\n" + "="*70)
print("RETRIEVAL PERFORMANCE COMPARISON")
print("="*70 + "\n")
print(results.to_string(index=False))
print("\n" + "="*70)

# highlight best performers
best_map = results.loc[results['MAP@10'].idxmax(), 'Method']
best_softmap = results.loc[results['SoftMAP'].idxmax(), 'Method']
best_ndcg = results.loc[results['nDCG@10'].idxmax(), 'Method']

print(f"\nBest MAP@10: {best_map}")
print(f"Best SoftMAP: {best_softmap}")
print(f"Best nDCG@10: {best_ndcg}")

improvement = ((results.loc[results['Method'] != 'TF-IDF (Baseline)', 'MAP@10'].max() - 
                results.loc[results['Method'] == 'TF-IDF (Baseline)', 'MAP@10'].values[0]) / 
               results.loc[results['Method'] == 'TF-IDF (Baseline)', 'MAP@10'].values[0] * 100)
print(f"\nImprovement over baseline: {improvement:.2f}%")


RETRIEVAL PERFORMANCE COMPARISON

             Method   MAP@10  SoftMAP  nDCG@10
  TF-IDF (Baseline) 0.293187 0.628017 0.355941
               BM25 0.343938 0.660747 0.402972
Semantic Embeddings 0.356935 0.681159 0.417468
       Hybrid (RRF) 0.364927 0.717927 0.429571


Best MAP@10: Hybrid (RRF)
Best SoftMAP: Hybrid (RRF)
Best nDCG@10: Hybrid (RRF)

Improvement over baseline: 24.47%


## Summary

### Key Findings
- Successfully improved retrieval performance above the TF-IDF baseline
- BM25 provides better keyword matching through probabilistic ranking
- Semantic embeddings capture meaning beyond exact keyword matches  
- Hybrid approach leverages strengths of both methods

### Method Explanations

**BM25**: An improvement over TF-IDF that uses probabilistic ranking. It handles term frequency saturation better (repeated terms have diminishing returns) and incorporates document length normalization. Well-suited for product search where queries often contain specific product terms.

**Semantic Embeddings**: Uses the sentence-transformers library with the 'all-MiniLM-L6-v2' model to generate dense vector representations. This captures semantic similarity - for example, "couch" and "sofa" are treated as similar even though they don't share terms. Useful when users search with synonyms or related concepts.

**Hybrid RRF**: Reciprocal Rank Fusion combines rankings from multiple methods without needing to normalize scores. Each method votes for documents, and RRF aggregates these votes using the formula: `score = Σ 1/(k + rank)`. This is more robust than simple score averaging and leverages complementary strengths of keyword and semantic approaches.
